# 0.0 Prepare Text Data

In [ ]:
!pip install nltk
!pip install gensim 

In [ ]:
!pip install pyldavis

In [16]:
import numpy as np
import pandas as pd
import xlrd # to read excel
import nltk # NLP toolkit
import re 
import json
import glob
import matplotlib.pyplot as plt # for visualization
import string # for handling string
import seaborn as sns
#Gensim
import gensim
#spacy 
import spacy
#vis
import pyLDAvis

In [18]:
summary = pd.read_excel('dataset.xlsx') 
new = summary.drop(['Date', 'UID', 'PDF URL', 'Cyber_Risk', 'Not_Cyber'], axis=1) 
' '.join(new['Summary'].tolist())
new['summary_in_lowercase']=new['Summary'].apply(lambda x: x.lower())
new['summary_without_punkt']=new['summary_in_lowercase'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

In [19]:
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stoplist = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zhiruoz2\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhiruoz2\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [20]:
from nltk.tokenize import word_tokenize 

# tokenization
def preprocess(text):
  formatted_text = text.lower()
  tokens = []
  for token in nltk.word_tokenize(formatted_text):
    tokens.append(token)
  tokens = [word for word in tokens if word not in stoplist and word not in string.punctuation]
  formatted_text1 = ' '.join(element for element in tokens)
  formatted_text2 = ''.join([i for i in formatted_text1 if not i.isdigit()])
  return formatted_text2

# remove stop words
new['summary_without_stopw']=new['summary_without_punkt'].apply(lambda x: preprocess(x))

# remove numbers

new['summary_without_num']=new['summary_without_stopw'].apply(lambda x: x.replace(r'\d+',''))

# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# Lemmatization with stopwords removal
new['lemmatized']=new['summary_without_num'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
new_text = ' '.join(new['lemmatized'].tolist())

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [26]:
def print_top_words(model, feature_names, n_top_words):
    tword = []
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        topic_w = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        tword.append(topic_w)
        print(topic_w)
    return tword

In [27]:
n_features = 1000 #Grab 1000 characteristic words
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(new.lemmatized)

In [28]:
n_topics = 8
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                learning_method='batch',
                                learning_offset=50,
#                                 doc_topic_prior=0.1,
#                                 topic_word_prior=0.01,
                               random_state=0)
lda.fit(tf)

LatentDirichletAllocation(learning_offset=50, max_iter=50, n_components=8,
                          random_state=0)

In [29]:
n_top_words = 25
tf_feature_names = tf_vectorizer.get_feature_names()
topic_word = print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
model graph problem network algorithm propose dynamic result method approach time risk base metric analysis study cybersecurity provide number consider distribution node value function different
Topic #1:
cyber defense attack game attacker model strategy threat defender agent policy learn deception framework information decision action optimal approach mechanism dynamic defensive design intelligence adversarial
Topic #2:
cybersecurity research cyber risk study provide technology information challenge present review tool software design threat develop identify development new discuss impact result increase work issue
Topic #3:
attack network vulnerability traffic intrusion detection attacker threat cyberattack cyber detect target malicious propose environment infrastructure exploit result botnet identify scenario flow work approach robot
Topic #4:
datum privacy cloud user propose service information secure data encryption access provide application blockchain key scheme device

In [30]:
topics=lda.transform(tf)

In [33]:
topic = []
for t in topics:
    topic.append("Topic #"+str(list(t).index(np.max(t))))
new['# of Topic with the highest probability']=topic
new['probability of each topic']=list(topics)
new.to_excel("data_topic.xlsx",index=False)

In [34]:
import pyLDAvis.sklearn

In [35]:
pyLDAvis.enable_notebook()
pic = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.display(pic)
pyLDAvis.save_html(pic, 'lda_pass'+str(n_topics)+'.html')
pyLDAvis.display(pic)

C:\Users\zhiruoz2\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
